<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 1106, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 1106 (delta 23), reused 253 (delta 4), pack-reused 832
Receiving objects: 100% (1106/1106), 183.42 MiB | 20.71 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Checking out files: 100% (882/882), done.


In [2]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย','ผักขึ้นช่าย','ผักชะอม','ผักชี','ผักชีฝรั่ง','ผักตำลึง',
               'มะกรูด','มะเขือพวง','มะเขือเทศ',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด','พุทรา','เพกา',
               'ฝรั่ง',
               'ฟัก','ฟักทอง',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [3]:
import pandas as pd
from glob import glob
import librosa

for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [4]:
df = pd.DataFrame.from_dict(source)
df

,audio_time,sampling_rate,feature_extraction,label,description
0,"[0.00015258789, 0.00022888184, 0.00022888184, ...",22050,"[[-638.9144, -641.23175, -641.37787, -636.0402...",0,กระชาย
1,"[-0.0009765625, -0.001449585, -0.001373291, -0...",22050,"[[-646.73065, -649.3781, -651.3155, -651.3928,...",0,กระชาย
2,"[-0.0001373291, -0.00018310547, -4.5776367e-05...",22050,"[[-655.651, -653.5817, -654.66907, -655.0482, ...",0,กระชาย
3,"[-0.00024414062, -0.0002593994, -0.00022888184...",22050,"[[-650.5033, -649.7863, -647.1397, -648.9082, ...",0,กระชาย
4,"[-0.0002593994, -0.00022888184, -0.0002593994,...",22050,"[[-639.25616, -639.47345, -640.0064, -640.5056...",0,กระชาย
...,...,...,...,...,...
875,"[-0.00016784668, -0.00015258789, -0.0001983642...",22050,"[[-668.4974, -670.4946, -672.10724, -671.17737...",43,สับปะรด
876,"[-0.00018310547, -0.0002593994, -0.00022888184...",22050,"[[-666.57153, -663.4788, -662.98236, -663.7374...",43,สับปะรด
877,"[0.0, 0.0, 0.0, 3.0517578e-05, -4.5776367e-05,...",22050,"[[-644.81506, -644.89075, -644.7345, -643.6741...",43,สับปะรด
878,"[-0.00018310547, -0.000289917, -0.00022888184,...",22050,"[[-614.8528, -614.2113, -613.9889, -613.8524, ...",43,สับปะรด


In [5]:
import numpy as np
import pandas as pd

Test = pd.DataFrame(columns=source.keys())

# create Test model
for description in np.unique(df['description']):
    imp = df[df['description']==description].sample(5,random_state=1)
    Test = pd.concat([Test,imp], axis=0)

# create Train model
Train = df.drop(Test.index)

X_train, X_test = np.array(Train['feature_extraction'].to_list()), np.array(Test['feature_extraction'].to_list())
y_train, y_test = np.array(Train['label'].to_list()), np.array(Test['label'].to_list())

In [6]:
import tensorflow as tf
from tensorflow import keras 
np.random.seed(42)
tf.random.set_seed(42)

model = keras.Sequential()
model.add(keras.layers.Conv2D(128, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(128, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
21/21 [==============================] - 30s 45ms/step - loss: 28.5681 - accuracy: 0.0258
Epoch 2/100
21/21 [==============================] - 1s 34ms/step - loss: 3.6748 - accuracy: 0.0500
Epoch 3/100
21/21 [==============================] - 1s 34ms/step - loss: 3.2130 - accuracy: 0.1576
Epoch 4/100
21/21 [==============================] - 1s 34ms/step - loss: 2.6255 - accuracy: 0.2591
Epoch 5/100
21/21 [==============================] - 1s 34ms/step - loss: 1.9787 - accuracy: 0.4439
Epoch 6/100
21/21 [==============================] - 1s 34ms/step - loss: 1.6215 - accuracy: 0.5167
Epoch 7/100
21/21 [==============================] - 1s 34ms/step - loss: 1.2190 - accuracy: 0.6303
Epoch 8/100
21/21 [==============================] - 1s 34ms/step - loss: 1.0596 - accuracy: 0.6788
Epoch 9/100
21/21 [==============================] - 1s 34ms/step - loss: 0.9536 - accuracy: 0.7152
Epoch 10/100
21/21 [==============================] - 1s 34ms/step - loss: 0.7903 - accuracy: 0.75

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 255, 128)       3328      
_________________________________________________________________
dropout (Dropout)            (None, 9, 255, 128)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 127, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 127, 128)       409728    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 127, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 63, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16128)             0

In [8]:
model.evaluate(X_test[:,:,:,None],y_test)

7/7 [==============================] - 1s 57ms/step - loss: 0.4394 - accuracy: 0.9318


[0.43939733505249023, 0.9318181872367859]

In [9]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(np.round(model.predict(X_test[:,:,:,None])),axis=1)
target_names = [f"Class {label}" for label in range(len(np.unique(source['label'])))]

print(classification_report(y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.62      1.00      0.77         5
     Class 1       0.80      0.80      0.80         5
     Class 2       0.83      1.00      0.91         5
     Class 3       1.00      0.80      0.89         5
     Class 4       1.00      1.00      1.00         5
     Class 5       1.00      1.00      1.00         5
     Class 6       1.00      1.00      1.00         5
     Class 7       1.00      1.00      1.00         5
     Class 8       1.00      0.80      0.89         5
     Class 9       1.00      1.00      1.00         5
    Class 10       1.00      0.80      0.89         5
    Class 11       1.00      1.00      1.00         5
    Class 12       1.00      0.80      0.89         5
    Class 13       1.00      0.80      0.89         5
    Class 14       1.00      1.00      1.00         5
    Class 15       0.57      0.80      0.67         5
    Class 16       1.00      1.00      1.00         5
    Class 17       0.83    